In [1]:
import pandas as pd
import airplanes.flight_passenger as f_p
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import holoviews as hv
from holoviews import opts
from airplanes.flight_passenger import get_airports_info
# _________________________________
import numpy as np
import hvplot.pandas
import panel as pn
import geoviews as gv 
import geopandas as gpd
import plotly.express as pex

In [2]:
pass_busy_file_path = 'data/passengers_busy_airports.csv'
pass_lbusy_file_path = 'data/passengers_less_busy_airports.csv'
flight_busy_file_path = 'data/flights_busy_airports.csv'
flight_lbusy_file_path = 'data/flights_less_busy_airports.csv'

In [3]:
pass_busy_data = f_p.process_data(pass_busy_file_path)
pass_lbusy_data = f_p.process_data(pass_lbusy_file_path)
flight_busy_data = f_p.process_data(flight_busy_file_path)
flight_lbusy_data = f_p.process_data(flight_lbusy_file_path)
pass_lbusy_data

,Year,Month,Domestic Passengers,International Passengers,Total Passengers,City,Quarter
0,2003,1,109323,4,109327,Boise,1
1,2003,2,102754,6,102760,Boise,1
2,2003,3,115814,4,115818,Boise,1
3,2003,4,97584,34,97618,Boise,2
4,2003,5,108447,81,108528,Boise,2
...,...,...,...,...,...,...,...
1229,2023,2,56729998,14933429,71663427,Sacramento,1
1230,2023,3,69709155,18782858,88492013,Sacramento,1
1231,2023,4,67572130,18816065,86388195,Sacramento,2
1232,2023,5,71412644,19833959,91246603,Sacramento,2


In [4]:
merge_by_col = ['Year', 'Month', 'Quarter', 'City']
busy_airports = f_p.merge_datasets(pass_busy_data, flight_busy_data, merge_by_col)
#_________#To do 
# busy_airports['Total Passengers'] = busy_airports['Total Passengers'].apply(lambda x : x / (10**4))
# busy_airports.rename(columns={'Total Passengers': 'Total Passengers (in 10^4)'}, inplace=True)
#_________#To do 
lbusy_airports = f_p.merge_datasets(pass_lbusy_data, flight_lbusy_data, merge_by_col)
busy_airports

,Year,Month,Quarter,City,Domestic Passengers,International Passengers,Total Passengers,Domestic Flights,International Flights,Total Flights
0,2003,1,1,Atlanta,2686976,184248,2871224,33041,1760,34801
1,2003,2,1,Atlanta,2554385,156427,2710812,29104,1517,30621
2,2003,3,1,Atlanta,3044047,181394,3225441,33096,1708,34804
3,2003,4,2,Atlanta,2836458,172833,3009291,31552,1651,33203
4,2003,5,2,Atlanta,2992648,190182,3182830,32354,1639,33993
...,...,...,...,...,...,...,...,...,...,...
1229,2023,2,1,Seattle,1381277,82372,1463649,12287,686,12973
1230,2023,3,1,Seattle,1672443,101305,1773748,14067,753,14820
1231,2023,4,2,Seattle,1689668,100579,1790247,14117,822,14939
1232,2023,5,2,Seattle,1830820,105173,1935993,15253,806,16059


In [5]:
lbusy_airports

,Year,Month,Quarter,City,Domestic Passengers,International Passengers,Total Passengers,Domestic Flights,International Flights,Total Flights
0,2003,1,1,Boise,109323,4,109327,2455,0,2455
1,2003,2,1,Boise,102754,6,102760,2161,0,2161
2,2003,3,1,Boise,115814,4,115818,2408,0,2408
3,2003,4,2,Boise,97584,34,97618,2177,0,2177
4,2003,5,2,Boise,108447,81,108528,2175,0,2175
...,...,...,...,...,...,...,...,...,...,...
1229,2023,2,1,Sacramento,56729998,14933429,71663427,567721,104389,672110
1230,2023,3,1,Sacramento,69709155,18782858,88492013,655859,122266,778125
1231,2023,4,2,Sacramento,67572130,18816065,86388195,636183,121239,757422
1232,2023,5,2,Sacramento,71412644,19833959,91246603,664863,126101,790964


_______________________________________________________________________________________________

For the `busy_airports` dataset:

**Independent Variables (Key Dimensions):**
- 'Year'
- 'Quarter'
- 'City'

**Dependent Variables (Value Dimensions):**

- 'Domestic Passengers'
- 'International Passengers'
- 'Total Passengers'
- 'Domestic Flights'
- 'International Flights'
- 'Total Flights'


In [34]:
key_dims = ['Year', 'City'] #'Quarter'
value_dims = ['Total Passengers', 'Total Flights']
busy_airports_ds = hv.Dataset(busy_airports, kdims=key_dims, vdims=value_dims)
busy_airports_ds = busy_airports_ds.aggregate(function=np.mean)
busy_airports_ds

:Dataset   [Year,City]   (Total Passengers,Total Flights)

In [35]:
key_dims = ['Year', 'City'] #'Quarter'
value_dims = ['Total Passengers', 'Total Flights']
busy_airports_ds = hv.Dataset(busy_airports, kdims=key_dims, vdims=value_dims)
busy_airports_ds = busy_airports_ds.aggregate(function=np.mean)
busy_airports_ds
plot1 = busy_airports_ds.to(hv.Curve, 'Year', value_dims[0]).opts(tools = ['hover'])
plot2 = busy_airports_ds.to(hv.Curve, 'Year', value_dims[1]).opts(tools = ['hover'])
layout_bports = hv.Layout(plot1 + plot2).cols(1)
layout_bports.opts(
    opts.Curve(width=700, height=250, framewise=True))
# #_________#To do 
# select = pn.widgets.Select(name='Select', groups={'Atlanta': [], 'less_busy': []})
# https://panel.holoviz.org/reference/widgets/Select.html

# select

:Layout
   .HoloMap.I  :HoloMap   [City]
      :Curve   [Year]   (Total Passengers)
   .HoloMap.II :HoloMap   [City]
      :Curve   [Year]   (Total Flights)

In [30]:
airports_inf = pd.read_csv('data/airports_information.csv')
airports_code = airports_inf['Airport Code'].tolist()
bars = busy_airports_ds.select(Year=(2012, 2023)).to(hv.Bars, key_dims, value_dims[0]).sort()
bars.opts(
    opts.Bars(width=980, height=500, tools=['hover'], xrotation=90, show_legend=False))

:Bars   [Year,City]   (Total Passengers)

In [31]:
grouped = busy_airports_ds.select(Year=(2012, 2022)).to(hv.Curve, 'Year', value_dims[1])
ndoverlay = grouped.overlay('City')
ndoverlay.opts(
    opts.Curve(width=900, height=600, color=hv.Cycle(values=['indianred', 'slateblue', 'lightseagreen', 'coral']), tools = ['hover']))

:NdOverlay   [City]
   :Curve   [Year]   (Total Flights)

_______________________________________________________________________________________________

# Airports Information 

In [12]:
get_airports_info()
# dom_pass = sum(pass_busy_data['Domestic Passengers'], pass_lbusy_data['Domestic Passengers'])

,Airport Code,Airport Name,City Name,Latitude,Longitude,City Name geo_name_id,Country Name geo_name_id,coordinates
0,DFW,Dallas/Fort Worth International,Dallas,32.896944,-97.038056,5722064,6252001.0,"32.896944, -97.038056"
1,OAK,Oakland International Airport,Oakland,37.716667,-122.216667,5378538,6252001.0,"37.716667, -122.216667"
2,SMF,Sacramento International,Sacramento,38.700000,-121.616667,5389489,6252001.0,"38.7, -121.616667"
3,MCI,Kansas City International Airport,Kansas City,39.291667,-94.730556,4393217,6252001.0,"39.291667, -94.730556"
4,JFK,John F Kennedy Intl,New York,40.633333,-73.783333,5128581,6252001.0,"40.633333, -73.783333"
5,PHL,Philadelphia International,Philadelphia,39.871944,-75.241111,5131095,6252001.0,"39.871944, -75.241111"
6,BOI,Boise Air Terminal/Gowen Field,Boise,43.564444,-116.222778,5586437,6252001.0,"43.564444, -116.222778"
7,SNA,John Wayne,Santa Ana,33.666667,-117.866667,5392900,6252001.0,"33.666667, -117.866667"
8,ATL,Hartsfield-jackson Atlanta International,Atlanta,33.636667,-84.428056,4883772,6252001.0,"33.636667, -84.428056"
9,SEA,Seattle-Tacoma International,Seattle,47.448889,-122.309444,5809844,6252001.0,"47.448889, -122.309444"


In [13]:
labels_p, labels_f = ['Domestic Flights', 'International Flights'], ['Domestic Flights', 'International Flights']
colors_p, colors_f = ['rgb(0, 128, 120)', 'rgb(235, 69, 0)'], ['rgb(31, 119, 180)', 'rgb(214, 39, 40)']
dom_bp, intl_bp = sum(pass_busy_data['Domestic Passengers']), sum(pass_busy_data['International Passengers']) #busy_pass


dom_lbp, intl_lbp = sum(pass_lbusy_data['Domestic Passengers']), sum(pass_lbusy_data['Domestic Passengers']) #lbusy_pass
lbusy_pass = dom_lbp + intl_lbp
lbusy_pass

27740091710

In [14]:
labels_p, labels_f = ['Domestic Passengers', 'International Passengers'], ['Domestic Flights', 'International Flights']
colors_p, colors_f = ['rgb(0, 128, 120)', 'rgb(235, 69, 0)'], ['rgb(214, 39, 40)', 'rgb(31, 119, 180)']

dom_pass = (busy_airports['Domestic Passengers'] + busy_airports['Domestic Passengers']).sum()

intl_pass = (busy_airports['International Passengers'] + busy_airports['International Passengers']).sum()

dom_flight = (busy_airports['Domestic Flights'] + busy_airports['Domestic Flights']).sum()

intl_flight = (busy_airports['International Flights'] + busy_airports['International Flights']).sum()

size_p, size_f = [dom_pass, intl_pass], [dom_flight, intl_flight]


fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]])

# Add traces for the donut charts
fig.add_trace(go.Pie(labels=labels_p, values=size_p, name=""),
              1, 1)

fig.add_trace(go.Pie(labels=labels_f, values=size_f, name=""),
              1, 2)

# Update traces and layout
fig.update_traces(hole=0.4, hoverinfo="label+percent+name", marker=dict(colors=colors_p), col=1)
fig.update_traces(hole=0.4, hoverinfo="label+percent+name", marker=dict(colors=colors_f), col=2)

title = "Percentage of Domestic and International Flights and Passengers"

fig.update_layout(
    title_text=title,
    annotations=[
        dict(text='', x=0.18, y=0.5, font_size=20, showarrow=False),
        dict(text='', x=0.82, y=0.5, font_size=20, showarrow=False)
    ]
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()